In [1]:
import os
import tf_agents.policies
import tf_agents.specs
from tf_agents.environments import suite_gym, parallel_py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.metrics import tf_metrics
from tf_agents.policies import policy_saver
from tf_agents.policies.tf_py_policy import TFPyPolicy
from tf_agents.replay_buffers import tf_uniform_replay_buffer, episodic_replay_buffer
from tf_agents.drivers import dynamic_episode_driver, dynamic_step_driver
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')  #  allows testing during training
from tf_agents.trajectories import time_step as ts, policy_step, trajectory
from reinforcement_learning import labeling_functions
labeling_function = labeling_functions['CartPole-v0']
from util.io.dataset_generator import map_rl_trajectory_to_vae_input
from util.io.dataset_generator import ErgodicMDPTransitionGenerator

In [2]:
py_env = suite_gym.load('CartPole-v0')
py_env.reset()
tf_env = tf_py_environment.TFPyEnvironment(py_env)
tf_env.time_step_spec()

TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=BoundedTensorSpec(shape=(4,), dtype=tf.float32, name='observation', minimum=array([-4.8000002e+00, -3.4028235e+38, -4.1887903e-01, -3.4028235e+38],
      dtype=float32), maximum=array([4.8000002e+00, 3.4028235e+38, 4.1887903e-01, 3.4028235e+38],
      dtype=float32)))

In [3]:
policy_dir = '../saves/CartPole-v0/policy/'

policy = tf.compat.v2.saved_model.load(policy_dir)
reward_metric = tf_metrics.AverageReturnMetric()
dynamic_episode_driver.DynamicEpisodeDriver(tf_env, policy, num_episodes=5,
                                            observers=[
                                                # display_safe_labeling,
                                                lambda _: py_env.render(mode='human'),
                                                reward_metric
                                            ]).run()
reward_metric.result()

<tf.Tensor: shape=(), dtype=float32, numpy=200.0>

In [7]:
import variational_mdp

vae_mdp = variational_mdp.load(
    '../../saves/CartPole-v0/models/vae_LS12_MC1_ER10.0-decay=0.00025-min=0_KLA0.0-growth=5e-06_TD0.67-0.50_1e-06-2e-06_seed=60421_params=relaxed_state_encoding-latent_policy/base',
    # '../../saves/CartPole-v0/models/vae_LS12_MC1_ER10.0-decay=0.00025-min=0_KLA0.0-growth=5e-06_TD0.67-0.50_1e-06-2e-06_seed=20421_params=relaxed_state_encoding-latent_policy/base',
    step=260000, discrete_action=True
)
print("VAE MDP loaded")

vae_mdp.eval_policy(eval_env=py_env, labeling_function=labeling_function, num_eval_episodes=20, render=True)

VAE MDP loaded
eval policy 200.0


<tf.Tensor: shape=(), dtype=float32, numpy=200.0>